In [12]:
import glob
import time
import cv2 as cv
import numpy as np
import cv2.aruco as aruco

In [13]:
cap = cv.VideoCapture('/home/mihir/rosbag2video/left_wall.mp4')
epochs = []
ref_pose = []

In [14]:
#marker length in meters
marker_length = 0.22
distortion_coeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
intrinsic_camera_mat = np.array([570.3422241210938, 0.0, 319.5, 0.0, 570.3422241210938, 239.5, 0.0, 0.0, 1.0])
rectification_matrix = np.array([1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0])
camera_projection_matrix = np.array([570.3422241210938, 0.0, 319.5, 0.0, 0.0, 570.3422241210938, 239.5, 0.0, 0.0, 0.0, 1.0, 0.0])
cpm = camera_projection_matrix.reshape(4,3)
icm = intrinsic_camera_mat.reshape(3,3)

In [15]:
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if (ret):
        # Our operations on the frame come here
        #gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
        parameters =  aruco.DetectorParameters_create()
    
        corners, ids, rejectedImgPoints = aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    
        if corners or ids:
            #timestamps = cap.get(cv.CAP_PROP_POS_MSEC)
            #time_list.append(timestamps)
            epochs.append(time.time())
            gray = aruco.drawDetectedMarkers(frame, corners)
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corners,marker_length,icm, distortion_coeffs)
            gray = aruco.drawAxis(gray, icm, distortion_coeffs, rvec, tvec, 0.1)

            #for bookeeping purposes
            pose_tuple = ids[0][0], tvec[0][0]
            print (pose_tuple)
            ref_pose.append(pose_tuple)
            
            
        # Display the resulting frame
        cv.imshow('frame',frame)
        if cv.waitKey(60) & 0xFF == ord('q'):
            break
        
                
    else:
        break
 
#release the capture once everything is done
cap.release()
cv.destroyAllWindows()

(1, array([1.7896217 , 0.13100506, 4.82302057]))
(1, array([1.78626527, 0.13397661, 4.63361509]))
(1, array([1.79797864, 0.12132473, 4.54016282]))
(1, array([1.79674782, 0.10534741, 4.45361792]))
(1, array([1.78942521, 0.09834355, 4.40192644]))
(1, array([1.83260551, 0.09810952, 4.47911929]))
(1, array([1.84800557, 0.09024537, 4.47837809]))
(1, array([1.80600327, 0.07943401, 4.317428  ]))
(1, array([1.78232983, 0.07604212, 4.23425164]))
(1, array([1.86709227, 0.07483794, 4.38076914]))
(1, array([1.90455079, 0.07415422, 4.45497765]))
(1, array([1.88271887, 0.0711604 , 4.39035932]))
(1, array([1.91236217, 0.07415436, 4.45497879]))
(1, array([1.82320015, 0.06120508, 4.23434035]))
(1, array([1.8624162 , 0.05672433, 4.31657467]))
(1, array([1.87006045, 0.06032513, 4.30380908]))
(1, array([1.87006045, 0.06032513, 4.30380908]))
(1, array([1.92290727, 0.05576608, 4.389943  ]))
(1, array([1.83215627, 0.05120072, 4.17456253]))
(1, array([1.86696836, 0.05015576, 4.24121852]))
(1, array([1.8398099

#### can also use time.clock but that will only give system time and not the actual time(12/24 hour format), with ros use ros.time.now

In [203]:
time.ctime(epochs[0])

'Wed Oct 24 14:34:27 2018'

In [ ]:
epochs

In [76]:
# write to a csv file just for testing 
import csv

with open('temp.csv', 'w') as csvfile:
    fieldnames = ['agent1', 'x1', 'y1', 'agent2', 'x2', 'y2', 'agent3', 'x3', 'y3']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for i in range(len(ref_pose)):
        #check the marker_id to decide when to put what
        if ref_pose[i][1][0] > 0:
            #puts data in the right most column if the object is on the right
            writer.writerow({'agent3': ref_pose[i][0], 'x3': ref_pose[i][1][0], 'y3': ref_pose[i][1][1]})
        else :
            #puts data in the left most column if the object is on the left
            writer.writerow({'agent1': ref_pose[i][0], 'x1': ref_pose[i][1][0], 'y1': ref_pose[i][1][1]})
    

In [109]:
#just to count the number of instances of each marker
marker17,marker32,marker40,marker25 = 0,0,0,0
for idx in range(len(ref_pose)):
    if ref_pose[idx][0] == 17:
        marker17+= 1
    elif ref_pose[idx][0] == 32:
        marker32+= 1
    elif ref_pose[idx][0] == 25:
        marker25+= 1
    elif ref_pose[idx][0] == 40:
        marker40+= 1

In [110]:
marker17

73

In [111]:
marker25

109

In [113]:
marker32

109

In [114]:
marker40

81

From the above analysis it can be observed that the number of times the markers were detected is not the same for two markers lying on either side of each other, the markers on the left wall(17,40) have relatively less number of detections when compared to the markers(32,25) on the right hand side of the wall. From the provided video we may deduce that this is due to the lighting conditions or due to the placement of the markers or also due to the field of view of the camera.

In the first case(17,32) both the markers were placed approximately at the same positions on the wall, the marker with id '32' was detected first this could have been due to the lighting conditions also notice that the detection of the marker with id 32 is more stable than the marker with id 17.

In the second case(40,25) the marker with id 40 is placed slightly lower than the marker with id 25, this results in it being detected first, but as the robot starts moving further along the corridor marker 40 goes out of the field of view before the marker 25 this may be attributed to the robot moving slightly more towards the right.

We may deduce from this that the optimal placement for the markers would be at a height slightly lower than the height at which the camera is placed or at a height which is approximately at the same level as the camera.

Also observed during the testing was that the qtcbcs realtions were observed only when the objects were in motion realtive to each other, but no qtcb realation is obeserved during the start and end of the video when the objects are realtively stationary, and hence no realtions can be calculated. During these instances we can use only the rcc or the distance calculus to approximate the motion of the robot.

In [5]:
np.array([9,254,1]) * icm

array([[5.13308002e+03, 0.00000000e+00, 3.19500000e+02],
       [0.00000000e+00, 1.44866925e+05, 2.39500000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])